In [ ]:
# ==========================================
# BLOQUE 1: Importación y Configuración
# ==========================================
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

#importación de dataset
df = pd.read_csv("https://raw.githubusercontent.com/munevardo/graduation_project/refs/heads/main/Data_Greener_all.csv")

# 1. Definimos las variables de entrada (Lo que sabe el modelo)
# 'Total' es el consumo global de la casa, más las variables de tiempo.
features_cols = ["Total", "hora", "DiaSemanaNum", "Mes"]

# 2. Definimos las cargas que queremos desagregar (Lo que va a predecir)
targets_list = ["Ventilation", "Sokets plug", "Lighting", "Other electricity", "Cooling", "Heating"]

# 3. Configuración de la RNN
TIME_STEPS = 24  # El modelo mirará las últimas 24 horas para decidir
EPOCHS = 15      # Cuántas veces repasa los datos (puedes subirlo a 30-50 para mejorar)
BATCH_SIZE = 64  # Cuántos datos procesa de golpe

# Lista vacía para guardar los resultados finales de cada carga
results_list = []

print("✅ Configuración lista. Targets a analizar:", targets_list)

✅ Configuración lista. Targets a analizar: ['Ventilation', 'Sokets plug', 'Lighting', 'Other electricity', 'Cooling', 'Heating']


In [ ]:
# Normalizar variables

# 1. Convertir Time a datetime
df["Time"] = pd.to_datetime(df["Time"], errors="coerce")

# 2. Convertir las demás columnas a numérico
cols_numericas = ["Ventilation", "Sokets plug", "Lighting", "Other electricity", "Cooling", "Heating"]
df[cols_numericas] = df[cols_numericas].apply(pd.to_numeric, errors="coerce")

# Verificar cambios
print(df.dtypes)

Time                 datetime64[ns]
Ventilation                 float64
Sokets plug                 float64
Lighting                    float64
Other electricity           float64
Cooling                     float64
Heating                     float64
dtype: object


In [ ]:
df["Time"] = pd.to_datetime(df["Time"], errors="coerce")

# Crear nuevas columnas
df['hora'] = df["Time"].dt.hour
df["Dia"] = df["Time"].dt.day
df["Mes"] = df["Time"].dt.month
df["Anno"] = df["Time"].dt.year
df["DiaSemanaNum"] = df["Time"].dt.weekday   # 0 = Lunes, 6 = Domingo
df["DiaSemana"] = df["Time"].dt.day_name()   # Nombre del día


df["Total"] = (
    df["Ventilation"]
    + df["Sokets plug"]
    + df["Lighting"]
    + df["Other electricity"]
    + df["Cooling"]
    + df["Heating"]
)

print(df.head())

                 Time  Ventilation  Sokets plug  Lighting  Other electricity  \
0 2017-01-03 13:00:00         27.4         41.8      53.7               89.6   
1 2017-01-03 14:00:00         21.6         37.5      50.9               64.6   
2 2017-01-03 15:00:00         18.5         37.4      60.1               64.2   
3 2017-01-03 16:00:00         29.7         38.0      52.6               43.2   
4 2017-01-03 17:00:00         16.7         38.1      56.7               42.4   

   Cooling  Heating  hora  Dia  Mes  Anno  DiaSemanaNum DiaSemana  Total  
0     18.6    498.0    13    3    1  2017             1   Tuesday  729.1  
1     61.9    500.0    14    3    1  2017             1   Tuesday  736.5  
2     16.5    480.0    15    3    1  2017             1   Tuesday  676.7  
3     19.8    390.0    16    3    1  2017             1   Tuesday  573.3  
4     15.2    373.0    17    3    1  2017             1   Tuesday  542.1  


## Paso 2: Función de Ventanas de Tiempo

Esta es la función auxiliar que transforma el dataset en "secuencias" tridimensionales que la **LSTM** puede entender.


In [ ]:
# ==========================================
# BLOQUE 2: Función Generadora de Secuencias
# ==========================================

def create_sequences(X, y, time_steps=TIME_STEPS):
    """
    Transforma datos planos en ventanas de tiempo.
    Entrada: (1000 datos, 5 variables)
    Salida:  (976 ventanas, 24 pasos de tiempo, 5 variables)
    """
    Xs, ys = [], []

    # Recorremos los datos dejando un margen para la ventana
    for i in range(len(X) - time_steps):
        # Guardamos la ventana de 'time_steps' horas
        Xs.append(X[i:(i + time_steps)])
        # Guardamos el valor objetivo justo después de la ventana
        ys.append(y[i + time_steps])

    return np.array(Xs), np.array(ys)

print("✅ Función de secuencias definida correctamente.")

✅ Función de secuencias definida correctamente.


## Paso 3: El Bucle de entrenamiento por categoría

Empezamos a Itera automáticamente por cada categoría de desagregación, prepara sus datos específicos, entrena una red nueva desde cero y evalúa su rendimiento.

In [ ]:
# ==========================================
# BLOQUE 3: Bucle de Entrenamiento Automatizado
# ==========================================

print("🚀 Iniciando el análisis masivo de cargas...\n")

for target_col in targets_list:
    print(f"🔄 Procesando carga: {target_col.upper()}...")

    # ---------------------------------------------------------
    # A. PREPARACIÓN DE DATOS ESPECÍFICA
    # ---------------------------------------------------------
    # Filtramos solo las features + la columna objetivo actual
    data = df[features_cols + [target_col]].values

    # Escalamos todo entre 0 y 1 (Vital para redes neuronales)
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data)

    # Separamos Entradas (X) y Salida (y) dentro de la matriz escalada
    # X = Todas las columnas menos la última
    # y = La última columna (la carga actual)
    X_scaled = data_scaled[:, :-1]
    y_scaled = data_scaled[:, -1]

    # ---------------------------------------------------------
    # B. DIVISIÓN CRONOLÓGICA (NO ALEATORIA)
    # ---------------------------------------------------------
    # Entrenamos con el 80% del pasado, probamos con el 20% del futuro
    train_size = int(len(data_scaled) * 0.8)

    # Datos crudos divididos
    X_train_raw, X_test_raw = X_scaled[:train_size], X_scaled[train_size:]
    y_train_raw, y_test_raw = y_scaled[:train_size], y_scaled[train_size:]

    # Convertimos a secuencias 3D usando la función del Paso 2
    X_train, y_train = create_sequences(X_train_raw, y_train_raw, TIME_STEPS)
    X_test, y_test = create_sequences(X_test_raw, y_test_raw, TIME_STEPS)

    # ---------------------------------------------------------
    # C. ARQUITECTURA DE LA RED NEURONAL
    # ---------------------------------------------------------
    # Limpiamos la memoria de Keras para no mezclar modelos anteriores
    tf.keras.backend.clear_session()

    model = Sequential()
    # Capa 1: LSTM que devuelve secuencias (para que la siguiente capa entienda el tiempo)
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2)) # Apaga neuronas al azar para evitar memorización (overfitting)

    # Capa 2: LSTM que comprime la información
    model.add(LSTM(units=30, return_sequences=False))
    model.add(Dropout(0.2))

    # Capa de Salida: Predice 1 solo valor (Watts)
    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    # ---------------------------------------------------------
    # D. ENTRENAMIENTO
    # ---------------------------------------------------------
    # verbose=0 hace que no imprima todas las líneas de entrenamiento para no ensuciar la pantalla
    model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.1, verbose=0, shuffle=False)

    # ---------------------------------------------------------
    # E. PREDICCIÓN Y DES-ESCALADO
    # ---------------------------------------------------------
    y_pred = model.predict(X_test, verbose=0)

    # Truco para invertir el escalado:
    # El scaler espera todas las columnas original, así que creamos matrices "dummy" (falsas)

    # 1. Des-escalar Predicciones
    dummy_pred = np.zeros((len(y_pred), data.shape[1])) # Matriz vacía del tamaño original
    dummy_pred[:, -1] = y_pred.flatten()                # Ponemos nuestra predicción en la última columna
    pred_inverse = scaler.inverse_transform(dummy_pred)[:, -1] # Transformamos y recuperamos solo la columna final

    # 2. Des-escalar Datos Reales (Para comparar peras con peras)
    dummy_real = np.zeros((len(y_test), data.shape[1]))
    dummy_real[:, -1] = y_test
    y_test_inverse = scaler.inverse_transform(dummy_real)[:, -1]

    # ---------------------------------------------------------
    # F. CÁLCULO DE MÉTRICAS Y GUARDADO
    # ---------------------------------------------------------
    rmse = np.sqrt(mean_squared_error(y_test_inverse, pred_inverse))
    r2 = r2_score(y_test_inverse, pred_inverse)

    # Guardamos en la lista
    results_list.append({
        "Carga": target_col,
        "RMSE (Watts)": round(rmse, 2),
        "R2 Score": round(r2, 4)
    })

    print(f"   ---> Terminado. R²: {r2:.4f} | Error (RMSE): {rmse:.2f} W\n")

print("🏁 ¡Análisis de todas las cargas completado!")

🚀 Iniciando el análisis masivo de cargas...

🔄 Procesando carga: VENTILATION...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


   ---> Terminado. R²: 0.2541 | Error (RMSE): 33.73 W

🔄 Procesando carga: SOKETS PLUG...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


   ---> Terminado. R²: 0.0278 | Error (RMSE): 47.90 W

🔄 Procesando carga: LIGHTING...


...

   ---> Terminado. R²: -12.7106 | Error (RMSE): 135.59 W

🔄 Procesando carga: COOLING...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


   ---> Terminado. R²: 0.1270 | Error (RMSE): 53.48 W

🔄 Procesando carga: HEATING...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


   ---> Terminado. R²: 0.1339 | Error (RMSE): 107.28 W

🏁 ¡Análisis de todas las cargas completado!
